<a href="https://colab.research.google.com/github/Duke-BC-DL-for-Energy-Infrastructure/Colab-Notebooks-for-Training-Models/blob/main/Cross_Domain_Train_Farmland_Val_Desert_Wind_Turbine_Object_Detection_with_Ultralytics_Yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import glob
import torch
import os

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.7.0+cu101 CPU


In [2]:
!git clone https://github.com/Duke-BC-DL-for-Energy-Infrastructure/yolov3.git  # clone our fork of the ultralytics repo

Cloning into 'yolov3'...
remote: Enumerating objects: 9644, done.
remote: Total 9644 (delta 0), reused 0 (delta 0), pack-reused 9644
Receiving objects: 100% (9644/9644), 8.20 MiB | 26.01 MiB/s, done.
Resolving deltas: 100% (6576/6576), done.


In [3]:
!mkdir -p data/

# get our data and unzip it
!wget -nc -O data/images.zip https://duke.box.com/shared/static/gc6g6054alnjtp0isdcccuxdr5wl59xy.zip
!wget -nc -O data/labels.zip https://duke.box.com/shared/static/nawqed2wofslp2dw5d9j17i35mmb0ul5.zip

!unzip -n -q data/images.zip -d data
!unzip -n -q data/labels.zip -d data

# get the synthetic images and labels and unzip those into the same directory
!wget -nc -O data/synthetic_images.zip https://duke.box.com/shared/static/51kby1zj73y5bj86pwogd6ma88p5x4sm.zip
!wget -nc -O data/synthetic_labels.zip https://duke.box.com/shared/static/4xw53kjtacxq21npnc6eb2q65xwcnt8v.zip

!unzip -n -q data/synthetic_images.zip -d data
!unzip -n -q data/synthetic_labels.zip -d data

#get the new labelled images and labels and unzip them
!wget -nc -O data/images_new.zip https://duke.box.com/shared/static/9hc0uqxric3gptb6j92fzswcuux4ou73.zip
!wget -nc -O data/label_new.zip https://duke.box.com/shared/static/41r1grgakpjswnzu274xk8k1edo9xfkd.zip
    
!unzip -n -q data/images_new.zip -d data
!unzip -n -q data/label_new.zip -d data

--2021-01-26 18:26:10--  https://duke.box.com/shared/static/gc6g6054alnjtp0isdcccuxdr5wl59xy.zip
Resolving duke.box.com (duke.box.com)... 107.152.24.197
Connecting to duke.box.com (duke.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/gc6g6054alnjtp0isdcccuxdr5wl59xy.zip [following]
--2021-01-26 18:26:11--  https://duke.box.com/public/static/gc6g6054alnjtp0isdcccuxdr5wl59xy.zip
Reusing existing connection to duke.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://duke.app.box.com/public/static/gc6g6054alnjtp0isdcccuxdr5wl59xy.zip [following]
--2021-01-26 18:26:11--  https://duke.app.box.com/public/static/gc6g6054alnjtp0isdcccuxdr5wl59xy.zip
Resolving duke.app.box.com (duke.app.box.com)... 107.152.24.201
Connecting to duke.app.box.com (duke.app.box.com)|107.152.24.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcl

In [4]:
# get modified cfg file that works for 1 class
!rm -rf yolov3/cfg 

!wget -nc -O yolov3/cfg.zip https://duke.box.com/shared/static/pq52dolw5lo5ds8fd7g9v72ekzlngekz.zip
!unzip -n -q yolov3/cfg.zip -d yolov3

--2021-01-26 18:36:37--  https://duke.box.com/shared/static/pq52dolw5lo5ds8fd7g9v72ekzlngekz.zip
Resolving duke.box.com (duke.box.com)... 107.152.24.197
Connecting to duke.box.com (duke.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/pq52dolw5lo5ds8fd7g9v72ekzlngekz.zip [following]
--2021-01-26 18:36:37--  https://duke.box.com/public/static/pq52dolw5lo5ds8fd7g9v72ekzlngekz.zip
Reusing existing connection to duke.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://duke.app.box.com/public/static/pq52dolw5lo5ds8fd7g9v72ekzlngekz.zip [following]
--2021-01-26 18:36:37--  https://duke.app.box.com/public/static/pq52dolw5lo5ds8fd7g9v72ekzlngekz.zip
Resolving duke.app.box.com (duke.app.box.com)... 107.152.24.201
Connecting to duke.app.box.com (duke.app.box.com)|107.152.24.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcl

In [5]:
# GET CROSS DOMAIN DATA
# delete the original data folder so we can unzip our own data folder with the wind turbine data
!rm -rf yolov3/data

# load in the data.zip file from the box
!wget -nc -O yolov3/data_cross_domain.zip https://duke.box.com/shared/static/ec7ykima36z399rbjuqwjouk72q5gb6p.zip
!unzip -n -q yolov3/data_cross_domain.zip -d yolov3

# get the synthetic_data file from the box
!wget -nc -O yolov3/data_cross_domain_adding_synthetic.zip https://duke.box.com/shared/static/lzzbb0cs2z79unegeyn4zc5hz1c9nw7a.zip
!unzip -n -q yolov3/data_cross_domain_adding_synthetic.zip -d yolov3

--2021-01-26 18:36:40--  https://duke.box.com/shared/static/ec7ykima36z399rbjuqwjouk72q5gb6p.zip
Resolving duke.box.com (duke.box.com)... 107.152.24.197
Connecting to duke.box.com (duke.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/ec7ykima36z399rbjuqwjouk72q5gb6p.zip [following]
--2021-01-26 18:36:40--  https://duke.box.com/public/static/ec7ykima36z399rbjuqwjouk72q5gb6p.zip
Reusing existing connection to duke.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://duke.app.box.com/public/static/ec7ykima36z399rbjuqwjouk72q5gb6p.zip [following]
--2021-01-26 18:36:40--  https://duke.app.box.com/public/static/ec7ykima36z399rbjuqwjouk72q5gb6p.zip
Resolving duke.app.box.com (duke.app.box.com)... 107.152.24.201
Connecting to duke.app.box.com (duke.app.box.com)|107.152.24.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcl

In [6]:
%pwd

'/content'

In [7]:
%cd yolov3

/content/yolov3


In [ ]:
# Training on farmland and validating on desert
!python3 train.py --cfg cfg/yolov3-spp.cfg --data data_cross_domain/wnd_px10_seed17.data --img-size 608 --epochs 2 --batch-size 10 --device 0

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
Namespace(adam=False, batch_size=10, bucket='', cache_images=False, cfg='cfg/yolov3-spp.cfg', data='data_cross_domain/wnd_px10_seed17.data', device='', epochs=2, evolve=False, freeze_layers=False, img_size=[608], multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/yolov3-spp-ultralytics.pt')
Using CPU

Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/
2021-01-26 18:36:58.523747: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Model Summary: 225 layers, 6.25733e+07 parameters, 6.25733e+07 gradients
Optimizer groups: 76 .bias, 76 Conv2d.weight, 73 other
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0

In [ ]:
!python3 test.py --cfg cfg/yolov3-spp.cfg --data data_cross_domain/wnd_px10_seed17.data --img-size 608 --weights weights/last.pt --conf-thres 0.3

In [ ]:
# Training on farmland and synthetic farmland. Validating on desert
!python3 train.py --cfg cfg/yolov3-spp.cfg --data data_cross_domain_adding_synthetic/wnd_syn_real_seed17.data --img-size 608 --epochs 2 --batch-size 10 --device 0

In [ ]:
!python3 test.py --cfg cfg/yolov3-spp.cfg --data data_cross_domain_adding_synthetic/wnd_syn_real_seed17.data --img-size 608 --weights weights/last.pt --conf-thres 0.3

In [ ]:
# Create folder for validation dataset, so that we can run detect.py on it

VAL_PATH = 'data_cross_domain/wnd_val_img_px10_seed17.txt' # Provide path to .txt file for validation set

from shutil import copyfile
%mkdir ../data/val

with open(VAL_PATH, 'r') as f:
  val_imgs =  [path.replace('\n', '') for path in f.readlines()]

for img in val_imgs:
  # copy image to val folder
  copyfile(img, img.replace('images', 'val'))

In [ ]:
# DETECT using last.pt
!python3 detect.py --names data_cross_domain/wnd.names --weights weights/last.pt --source ../data/val --img-size 608 --save-txt --conf-thres 0.3